In [19]:
import pickle
import os,json
from TmFunctions import *
from gensim.models import KeyedVectors
from scipy import spatial
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel


model = KeyedVectors.load_word2vec_format('F:/react-data/iconic/GoogleNews-vectors-negative300-SLIM.bin',binary=True)

In [24]:
def LoadInfo(photo,k):
    with open(f'F:/react-data/iconic/topicmodels/lda-{photo}-{k}-model.pkl','rb') as f:
        lda_model = pickle.load(f)
    
    id2word = corpora.Dictionary.load(f'F:/react-data/iconic/topicmodels/lda-{photo}-{k}-dictionary.pkl')
    
    with open(f'F:/react-data/iconic/topicmodels/lda-{photo}-{k}-corpus.json','r') as f:
        corpus = json.load(f)
    
    with open(f'F:/react-data/iconic/topicmodels/lda-{photo}-texts.json','r') as f:
        texts = json.load(f)
    
    return lda_model,id2word,corpus,texts

def ComputeCoherence(photo,k):
    lda_model,id2word,corpus,texts = LoadInfo(photo,k)
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

def LoadWords(photo,k):
    with open(f'F:/react-data/iconic/topicmodels/lda-{photo}-{k}-model.pkl','rb') as f:
        lda_model = pickle.load(f)
    topic_words = lda_model.show_topics(num_topics=k, num_words=25,formatted=False)
    col = []

    for topic in topic_words:
        list_ = ", ".join([x[0] for x in topic[1]])
        col.append(list_)
    return col

def CompareTopics(topic_1,topic_2,model):
    topic_1 = [w for w in topic_1.split(', ') if w in model.wv.vocab]
    topic_1_vector = np.average(model[topic_1], axis=0)
    
    topic_2 = [w for w in topic_2.split(', ') if w in model.wv.vocab]
    topic_2_vector = np.average(model[topic_2], axis=0)
    
    return 1 - spatial.distance.cosine(topic_1_vector, topic_2_vector)

In [22]:
list_photos = "AlanKurdi AbuGhraib Anasuma Berlin Camp Che ChildVulture ChileCoup FallingMan Ghandi Hindenburg IwoJima KentState ManMoon Mao MigrantMother Monk NapalmGirl Plane911 Rwanda SharbatGula SpanishSoldier TankMan TimesSquareKiss VietCong WarRoom".split(' ')

In [ ]:
dict_coh = {}

In [28]:
for photo in tqdm(list_photos[1:]):
    
    list_coh = []
    for k in [4,6,8,12,16]:
        
        coh = ComputeCoherence(photo,k)
        list_coh.append(coh)
    dict_coh.update({photo:list_coh})


100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [1:35:58<00:00, 251.17s/it]


In [30]:
dict_coh_bu = dict_coh

In [35]:
dict_coh

{'AlanKurdi': [0.4633460114432721,
  0.44935147186174224,
  0.5098089107099322,
  0.44693040700084047,
  0.4754181937033939],
 'AbuGhraib': [0.4051849922801247,
  0.40544823050440804,
  0.4333224054355822,
  0.41622173394151435,
  0.4326624062498588],
 'Anasuma': [0.4636261083191553,
  0.43263365221647204,
  0.48206890528868973,
  0.46693644253807803,
  0.4663491986900898],
 'Berlin': [0.40397562349171445,
  0.4525289279630373,
  0.46566453639152405,
  0.4782511326963599,
  0.47164985249752583],
 'Camp': [0.5011755438613124,
  0.46125994849292123,
  0.42839216045988565,
  0.4575073897630774,
  0.4255125566142951],
 'Che': [0.5053634055178202,
  0.5215236185844024,
  0.539950055523594,
  0.5226113579213205,
  0.48953360006570934],
 'ChildVulture': [0.40502914646315297,
  0.40264275505451746,
  0.40604972523746174,
  0.445103664276602,
  0.4480939706333502],
 'ChileCoup': [0.4246711805941946,
  0.4357490463568328,
  0.4353345708466677,
  0.3916305595749729,
  0.47106158788102226],
 'Fall

In [32]:
pd.DataFrame(dict_coh).to_csv('coherence-scores.csv',index=False)

In [43]:
d = pd.DataFrame(dict_coh,index=[f"{n} Topics" for n in [4,6,8,12,16]]).T
d = d.round(3)
d.to_csv('coherence-scores-2.csv')

\begin{tabular}{lrrrrr}
\toprule
{} &  4 Topics &  6 Topics &  8 Topics &  12 Topics &  16 Topics \\
\midrule
AlanKurdi       &  0.463346 &  0.449351 &  0.509809 &   0.446930 &   0.475418 \\
AbuGhraib       &  0.405185 &  0.405448 &  0.433322 &   0.416222 &   0.432662 \\
Anasuma         &  0.463626 &  0.432634 &  0.482069 &   0.466936 &   0.466349 \\
Berlin          &  0.403976 &  0.452529 &  0.465665 &   0.478251 &   0.471650 \\
Camp            &  0.501176 &  0.461260 &  0.428392 &   0.457507 &   0.425513 \\
Che             &  0.505363 &  0.521524 &  0.539950 &   0.522611 &   0.489534 \\
ChildVulture    &  0.405029 &  0.402643 &  0.406050 &   0.445104 &   0.448094 \\
ChileCoup       &  0.424671 &  0.435749 &  0.435335 &   0.391631 &   0.471062 \\
FallingMan      &  0.482592 &  0.529249 &  0.528790 &   0.496341 &   0.489210 \\
Ghandi          &  0.395990 &  0.395529 &  0.426028 &   0.438794 &   0.476425 \\
Hindenburg      &  0.524997 &  0.464444 &  0.482135 &   0.477668 &   0.470954 \\

In [ ]:
df_prob = pd.DataFrame()

for k,v in tp_prob.items():
    
    l = pd.DataFrame([[x,z] for x,z in v.items()])
    l['photo'] = k
    df_prob = df_prob.append(l)
df_prob.columns = ['topic','probability','photo']
df_prob_p = df_prob.pivot(index='photo',columns='topic',values='probability').fillna(0).reset_index()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(dmu)
principalDf = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2'])
principalDf['labels'] = list(dm.columns)

fig, ax = plt.subplots(figsize=(30,20))
ax.scatter(principalDf['pc1'], principalDf['pc2'])
samp = random.sample(list(range(208)),20)
for i, txt in enumerate(principalDf['labels']):
    if i in samp:
        ax.annotate(txt, (principalDf['pc1'][i], principalDf['pc2'][i]),fontsize=14)